### Passo 1 - Importar Arquivos e Bibliotecas

In [ ]:
import pandas as pd
from pathlib import Path

#Aqui estamos pré-visualizando os arquivos que serão analisados
vendas_df = pd.read_excel('Vendas.xlsx')
email_df = pd.read_excel('Emails.xlsx')
lojas_df = pd.read_csv('Lojas.csv',encoding='latin1', sep=';')
#Sempre importante durante o processo visualizar o atual estado do seu código 
display(vendas_df.head())
display(email_df.head())
display(lojas_df.head())

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [ ]:
#Mesclando dois dataframe a partir do ID 
vendas_df = vendas_df.merge(lojas_df, on='ID Loja')
display(vendas_df)

In [ ]:
#Aqui criando um dicionario vario para guardar os nomes de todas as lojas
grupo_lojas = {}
#Percorrendo o Dataframe lojas_df
for loja in lojas_df['Loja']:
    grupo_lojas[loja] = vendas_df.loc[vendas_df['Loja'] == loja, :]
display(grupo_lojas['Iguatemi Campinas'])
display(grupo_lojas['Shopping Recife'])    

In [ ]:
#Estabelendo o dia indicador como sendo a maior data em vendas_df
dia_indicador = vendas_df['Data'].max()
print(dia_indicador)
print(f'Rearranjando o formato da data {dia_indicador.day}/{dia_indicador.month}/{dia_indicador.year} ')

### Passo 3 - Salvar a planilha na pasta de backup
#### Nessa etapa fazemos a verificação da existencia de uma pasta da devida loja com o seu respectivo arquivo, caso não exista criamos se existir criamos o arquivo

In [ ]:
backup_arquivo = Path(r'Backup Arquivos Lojas')
arquivos_lojas = backup_arquivo.iterdir()
lista_nomes_lojas = []

for arquivo in arquivos_lojas:
    lista_nomes_lojas.append(arquivo.name)

for loja in grupo_lojas:
    if loja not in lista_nomes_lojas:    
        nova_pasta = backup_arquivo / loja
        nova_pasta.mkdir()
    
    nome_arquivo = f"{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx"
    local_arquivo = backup_arquivo / loja / nome_arquivo
    
    grupo_lojas[loja].to_excel(local_arquivo)


### Passo 4 - Criando as variaveis de cada indicador a ser analisado

In [ ]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

#### Dica fazer o calculo dos indicadores primeiro para uma loja depois usar uma estrutura de repetição para todas as outras

In [ ]:
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders

#Calculando os indicadores
for loja in grupo_lojas:
  vendas_loja = grupo_lojas[loja]
  vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

  #faturamento
  faturamento_ano = vendas_loja['Valor Final'].sum()
  #print(faturamento_ano)
  faturamento_dia = vendas_loja_dia['Valor Final'].sum()
  #print(faturamento_dia)

  #diversidade de produtos
  qtde_produtos_ano = len(vendas_loja['Produto'].unique())
  #print(qtde_produtos_ano)
  qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
  #print(qtde_produtos_dia)

  #ticket medio
  valor_venda = vendas_loja.groupby('Código Venda').sum()
  ticket_medio_ano = valor_venda['Valor Final'].mean()
  #print(ticket_medio_ano)

  #ticket_medio_dia
  valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
  ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
  #print(ticket_medio_dia)
  
  #enviar o e-mail
  fromaddr = "E-mail gmail"#Verifique as configurações de segurança do gmail
  toaddr = email_df.loc[email_df['Loja']==loja, 'E-mail'].values[0]#Use o arquivo com a lista de e-mails para tornar mais eficiente o trabalho
  nome = email_df.loc[email_df['Loja']==loja, 'Gerente'].values[0]
    
  msg = MIMEMultipart() 
  msg['From'] = fromaddr 
  msg['To'] = toaddr 
  msg['Subject'] = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

  if faturamento_dia >= meta_faturamento_dia:
      cor_fat_dia = 'green'
  else:
      cor_fat_dia = 'red'
  if faturamento_ano >= meta_faturamento_ano:
      cor_fat_ano = 'green'
  else:
      cor_fat_ano = 'red'
  if qtde_produtos_dia >= meta_qtdeprodutos_dia:
      cor_qtde_dia = 'green'
  else:
      cor_qtde_dia = 'red'
  if qtde_produtos_ano >= meta_qtdeprodutos_ano:
      cor_qtde_ano = 'green'
  else:
      cor_qtde_ano = 'red'
  if ticket_medio_dia >= meta_ticketmedio_dia:
      cor_ticket_dia = 'green'
  else:
      cor_ticket_dia = 'red'
  if ticket_medio_ano >= meta_ticketmedio_ano:
      cor_ticket_ano = 'green'
  else:
      cor_ticket_ano = 'red'
  
  body = f"""
  <p>Bom dia, {nome}</p>

  <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da Loja <strong>{loja}</strong> foi:</p>

  <table>
    <tr>
      <th>Indicador</th>
      <th>Valor Dia</th>
      <th>Meta Dia</th>
      <th>Cenário Dia</th>
    </tr>
    <tr>
      <td>Faturamento</td>
      <td style="text-align: center">R${faturamento_dia:.2f}</td>
      <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
      <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
    </tr>
    <tr>
      <td>Diversidade de Produtos</td>
      <td style="text-align: center">{qtde_produtos_dia}</td>
      <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
      <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
    </tr>
    <tr>
      <td>Ticket Médio</td>
      <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
      <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
      <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
    </tr>
  </table>
  <br>
  <table>
    <tr>
      <th>Indicador</th>
      <th>Valor Ano</th>
      <th>Meta Ano</th>
      <th>Cenário Ano</th>
    </tr>
    <tr>
      <td>Faturamento</td>
      <td style="text-align: center">R${faturamento_ano:.2f}</td>
      <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
      <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
    </tr>
    <tr>
      <td>Diversidade de Produtos</td>
      <td style="text-align: center">{qtde_produtos_ano}</td>
      <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
      <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
    </tr>
    <tr>
      <td>Ticket Médio</td>
      <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
      <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
      <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
    </tr>
  </table>
  <p>João Medeiros</p>
  """
  msg.attach(MIMEText(body, 'html')) 
  filename = f"{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx"
  attachment = open(r"Local do arquivo", "rb")
  p = MIMEBase('application', 'octet-stream') 
  p.set_payload((attachment).read()) 
  encoders.encode_base64(p) 

  p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  msg.attach(p) 
  s = smtplib.SMTP('smtp.gmail.com', 587) 
  s.starttls() 
  s.login(fromaddr, "diykpkcaoujaauoa") 
  text = msg.as_string() 
  s.sendmail(fromaddr, toaddr, text)

  print('E-mail da Loja {} enviado'.format(loja))

### Passo 7 - Criar ranking para diretoria

In [ ]:
faturamento_lojas = vendas_df.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_ano)

name_file = f'{dia_indicador.day}_{dia_indicador.month}_Ranking_anual.xlsx'
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(name_file))

vendas_dia = vendas_df.loc[vendas_df['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_dia)

name_file = f'{dia_indicador.day}_{dia_indicador.month}_Ranking_dia.xlsx'
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(name_file))

### Passo 8 - Enviar e-mail para diretoria

In [ ]:
#enviar o e-mail para Diretoria
fromaddr = "E-mail gmail"
toaddr = email_df.loc[email_df['Loja']=='Diretoria', 'E-mail'].values[0]#Lista da e-mail
  
msg = MIMEMultipart() 
msg['From'] = fromaddr 
msg['To'] = toaddr 
msg['Subject'] = f'Ranking dia {dia_indicador.day}/{dia_indicador.month}'


body = f"""
Prezados,

Bom dia,

Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com o Fatruramentor R$ {faturamento_lojas_dia.iloc[0, 0]:.2f}
Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com o Fatruramentor R$ {faturamento_lojas_dia.iloc[-1, 0]:.2f}

Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[0]} com o Fatruramentor R$ {faturamento_lojas_ano.iloc[0, 0]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com o Fatruramentor R$ {faturamento_lojas_ano.iloc[-1, 0]:.2f}

Segue anexo os Rankings do Dia e do Ano de todas as lojas.

Qualquer dúvida estou à disposição.

Atenciosamente,

João Medeiros
"""
msg.attach(MIMEText(body, 'plain')) 
filename = f"{dia_indicador.day}_{dia_indicador.month}_Ranking_anual.xlsx"
attachment = open(r"Local do arquivo", "rb")
p = MIMEBase('application', 'octet-stream') 
p.set_payload((attachment).read()) 
encoders.encode_base64(p) 

p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
msg.attach(p)

filename = f"{dia_indicador.day}_{dia_indicador.month}_Ranking_dia.xlsx"
attachment = open(r"Local do arquivo", "rb")
p = MIMEBase('application', 'octet-stream') 
p.set_payload((attachment).read()) 
encoders.encode_base64(p) 

p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
msg.attach(p)

s = smtplib.SMTP('smtp.gmail.com', 587) 
s.starttls() 
s.login(fromaddr, "diykpkcaoujaauoa") 
text = msg.as_string() 
s.sendmail(fromaddr, toaddr, text)

print('E-mail da Diretoria enviado')